## Load relevant packages

In [ ]:
# Import relevant packages
import dicom
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
%matplotlib inline

## Load cancer labels for patients

The file "stage1_labels.csv" is assumed to be in the same folder as the notebook.

In [ ]:
#================================================================================
# THIS PART IS AUTOMATIC
cancerlabels = pd.read_csv("./stage1_labels.csv", index_col=0)
print "We have a total of %d patients, %.2f%% of which were eventually diagnosed with cancer" %(cancerlabels.shape[0], 
                                                                   cancerlabels.cancer.mean()*100)

## Load image files

Each folder contains the images of a given patient; the images are loaded using the package dicom . 

All we need to do is specify the pathname for the folder in which we placed all the patients images.

In [ ]:
#================================================================================
# USER INPUT
imagesfolder_path = "./LungImages/Sample"

#================================================================================
# THIS PART IS AUTOMATIC
allfilepathnames = [[folderandfiles[0]+"/"+imname for imname in folderandfiles[2]] 
                          for folderandfiles in os.walk(imagesfolder_path) if folderandfiles[2]!=[]]

#allimagenames = [imname[:-4] for folderandfiles in os.walk("./LungImages/Sample") 
#                 for imname in folderandfiles[2] if folderandfiles[2]!=[]]

"allfilepathnames" has the structure: [*list of paths for first patient*, *list of paths for second patient*, ...]

where each list of paths is [*path of first photo*, *path of second photo*, ...]

So each element in allfilepathnames contains all images for that patient. The problem is that the ordering of these photos is pretty much random, i.e. not from top to bottom along the patient's chest.

We'll now load the files and order them correctly.

In [ ]:
# First we make a function that takes a list of file-pathnames, loads the images, and returns
# the loaded dicom files along with the patient's ID.
def getSlicesandID(listofpathnames):
    patientslices = np.array([dicom.read_file(pathname) for pathname in listofpathnames])
    # Each slice has a whole bunch of attributes belonging to the patient. 
    # N.B. the attribute PatientBirthDate is fake in this dataset.
    patientID = patientslices[0].PatientID
    
    # Now we reorder the slices according to the z-axis positions, given by the attribute SliceLocation
    correctordering = np.argsort([currentslice.SliceLocation for currentslice in patientslices])
    patientslices = patientslices[correctordering]
    
    return (patientslices,patientID)

### Resizing images

The images are large - 512x512 pixels. We also have many slices, several hundreds of them usually. We now make functions for making them all into a standard size.

*We assume the slices are evenly spaced out for each patient!*

In [ ]:
#================================================================================
# USER INPUT
imagesize_xy = (150, 150) # This is the number of pixels on each axis.
num_slices = 20

In order to resize them all we make a little function that does this. When selecting the number of slices in the vertical direction, we have to choose whether we want to simply select out a subset of slices (evenly spaced out to each other) or whether we want to take the average of a chosen number of vertical "blocks"; this second approach effectively increases the slice thickness so as to give you a total number of slices = num_slices.

In [ ]:
# This function takes a list of slices and returns a list of matrices describing the pixel-values,
# where the number and shape of matrices is determined by resize_pixels and num_vertical_images.
# We can choose method="average" of method="spaced".
def resizeImagesAndThickness(currentpatientslices, resize_pixels, num_vertical_images, method="average"):
    averagedistance = len(currentpatientslices) / float(num_vertical_images)
    # Now we'll turn this into a list of ranges that determine the positions participating in each chunk.
    chunkpositions = [[int(i*averagedistance), int((i+1)*averagedistance)] for i in range(num_vertical_images)]
    if method=="average":
        # We make lists of slices in each chunk
        slicechunks = [currentpatientslices[chunkpos[0]: chunkpos[1]] for chunkpos in chunkpositions]
        # For each slice in each chunk, we use the attribute .pixel_array to get the pixels-matrix.
        # We then resize this matrix using opencv, and take the mean of all matrices.
        finalimages = [np.mean([cv2.resize(theslice.pixel_array, resize_pixels) for theslice in chunk], axis=0) 
                     for chunk in slicechunks]
    if method=="spaced":
        # We'll only select the middle element in each chunk.
        middlepoint_slices = [currentpatientslices[int(np.mean(chunkpos))] for chunkpos in chunkpositions]
        finalimages = [cv2.resize(theslice.pixel_array, resize_pixels) for theslice in middlepoint_slices]
    return finalimages

Now we'll finally put it all together and extract the matrices (repesenting slices) for each patient.

In [ ]:
#================================================================================
# USER INPUT
slice_thickness_method = "average" # Have to choose either "average" or "spaced"

#================================================================================
# THIS PART IS AUTOMATIC
# (N.B. This part of the code would be easy to parallelize)

allpatients_slicematrices = []
allpatients_ID = []

for folder in allfilepathnames:
    (patientslices,patientID) = getSlicesandID(folder)
    slicematrices = resizeImagesAndThickness(patientslices, imagesize_xy, num_slices, method=slice_thickness_method)
    
    allpatients_ID.append(patientID)
    allpatients_slicematrices.append(slicematrices)
# It's nice to have them in numpy array format
allpatients_slicematrices = np.array(allpatients_slicematrices)
allpatients_ID = np.array(allpatients_ID)

Now we have all image files for all patients!

## Save the matrices to file

In [ ]:
np.save('allpatients_slicematrices.npy', allpatients_slicematrices)

## Draw slices

We'll now draw the slices for a few patients, just to see aht they look like. We'll make a little function that does this.

In [ ]:
# The insput is a list of matrices describing the pixels. The matrices are not required to all have the same resolution.
def drawSlices(list_of_image_matrices):
    # We'll make a figure with subplots in it
    numberofrows = int(np.ceil(len(list_of_image_matrices) / 6.))
    fig, axes = plt.subplots(nrows=numberofrows, ncols=6, figsize=(12,2*numberofrows))
    # Now on each axis we can draw the slice
    for (currentax, currentimage) in zip(axes.ravel(), list_of_image_matrices):
        currentax.imshow(currentimage, cmap="gray")
        # The ticks are useless and ugly
        currentax.set_xticks([])
        currentax.set_yticks([])
    # Finally we remove those plots that have nothing in them
    for remainingax in axes.ravel()[len(list_of_image_matrices):]:
        remainingax.axis("off")
    return fig, axes

In [ ]:
print "Patient without cancer"
drawSlices(allpatients_slicematrices[0]);

In [ ]:
print "Patient with cancer"
drawSlices(allpatients_slicematrices[4]);